<a href="https://colab.research.google.com/github/abdyraman/harassmentsurvey/blob/main/HarassmentSurvey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#Authentification to google collab
from google.colab import auth
auth.authenticate_user()
#Other libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

In [16]:
import gspread
from google.auth import default
creds,_=default()

gs=gspread.authorize(creds)
#Get google sheet
# Open the Google Sheet by name (do not include the .gsheet extension)
sheet = gs.open("Harassment in the Workplace (Responses)") # Correct name without .gsheet

# Access the specific worksheets
df_text = pd.DataFrame(sheet.worksheet("Form Responses 1").get_all_records())

In [17]:
#Checking the unique answer points per feature
unique_counts_objects = df_text.select_dtypes('object').nunique()

# Looping through each categorical variable and printing its unique values and counts
for i in unique_counts_objects.index:
    unique_values = df_text[i].value_counts()
    print(f'Unique values of {i}:')
    print(unique_values)
    print()

Unique values of Timestamp:
Timestamp
10/14/2024 11:54:38    1
12/17/2024 11:54:38    1
12/27/2024 11:54:38    1
12/26/2024 11:54:38    1
12/25/2024 11:54:38    1
                      ..
11/14/2024 11:54:38    1
11/13/2024 11:54:38    1
11/12/2024 11:54:38    1
11/11/2024 11:54:38    1
1/22/2025 11:54:38     1
Name: count, Length: 101, dtype: int64

Unique values of Have you received any employer-sponsored training on how to handle potentially violent or harassment situations?:
Have you received any employer-sponsored training on how to handle potentially violent or harassment situations?
Yes      36
No       35
Other    30
Name: count, dtype: int64

Unique values of If yes, has your training prepared you to deal with violent situations that may arise in your working environment?:
If yes, has your training prepared you to deal with violent situations that may arise in your working environment?
I don't know    38
Yes             33
No              30
Name: count, dtype: int64

Unique v

In [21]:
def clean_column_names(columns):
    # Define the patterns to remove
    patterns = [
        r"Please rate the following statements on a scale from 'Strongly Disagree' to 'Strongly Agree' \s*\[",
        r"Please rate the following statements on a scale from 'Never' to 'All of the time'. \s*\[",
    ]

    # Join patterns with `|` for regex
    combined_pattern = '|'.join(patterns)

    # Perform the replacement and ensure spaces are handled correctly
    cleaned_columns = columns.str.replace(combined_pattern, "", regex=True).str.strip()

    # Remove any leading `[` and trailing `]`
    cleaned_columns = cleaned_columns.str.replace(r'^\s*\[', '', regex=True).str.replace(r'\s*\]', '', regex=True)

    # Normalize spaces to a single space, only if needed
    cleaned_columns = cleaned_columns.str.replace(r'\s+', ' ', regex=True)

    return cleaned_columns


In [22]:
# Select object columns from df_text
text = df_text.select_dtypes(include=['object'])
# # Clean the column names
text.columns = clean_column_names(text.columns)

In [23]:
text.columns

Index(['Timestamp',
       'Have you received any employer-sponsored training on how to handle potentially violent or harassment situations?',
       'If yes, has your training prepared you to deal with violent situations that may arise in your working environment?',
       'Have you been trained to recognize and address discrimination, stereotypes, domestic violence and other aspects of human rights-related harassment and violence?',
       'Is there a violence and harassment prevention program at your workplace?',
       'Does your workplace have a written policy concerning violence and harassment?',
       'The organization's policies and processes with respect to prohibiting and reporting inappropriate gender-related behavior are easy to understand and follow',
       'Star performers are held to the same standards as other employees with respect to inappropriate gender-related behavior',
       'I would be comfortable reporting inappropriate conduct by a supervisor',
       'I wou

In [27]:
behavior_interactions=text[["The organization's policies and processes with respect to prohibiting and reporting inappropriate gender-related behavior are easy to understand and follow"
,"Star performers are held to the same standards as other employees with respect to inappropriate gender-related behavior"
,"I would be comfortable reporting inappropriate conduct by a supervisor"
,"I would be comfortable reporting inappropriate sexual conduct by a coworker"
,"There are adverse consequences for senior leaders who are abusive, disrespectful, or hostile"
,"In case I experience or witness harassment in the work environment, I have an action plan on how to report"
,"I am confident that complaints of harassment will be taken seriously and investigated thoroughly"
,"There are adequate confidential channels for reporting harassment"
,"The current measures and procedures for preventing and addressing harassment are effective"
,"The organization's culture promotes respect and safety for all employees"
,"Senior leadership demonstrates a commitment to preventing and addressing harassment"
,"The training programs address the nuances of different types of harassment and discrimination"]]

# Melt the DataFrame to long format
melted_values = behavior_interactions.melt(var_name='Statement', value_name='Rating')

# Count occurrences of each rating per statement
rating_counts_behavior_interactions = melted_values.groupby(['Statement','Rating']).size().reset_index(name='Count')

In [31]:
'Strongly Agree'=5, 'Agree'=4, 'Neutral'=3, 'Disagree'=2,'Strongly Disagree'=1

,Statement,Rating,Count
0,I am confident that complaints of harassment w...,Agree,20
1,I am confident that complaints of harassment w...,Disagree,21
2,I am confident that complaints of harassment w...,Neutral,16
3,I am confident that complaints of harassment w...,Strongly Agree,23
4,I am confident that complaints of harassment w...,Strongly Disagree,21
5,I would be comfortable reporting inappropriate...,Agree,15
6,I would be comfortable reporting inappropriate...,Disagree,21
7,I would be comfortable reporting inappropriate...,Neutral,21
8,I would be comfortable reporting inappropriate...,Strongly Agree,20
9,I would be comfortable reporting inappropriate...,Strongly Disagree,24


In [29]:
biases=text[["I believe my opportunities for career success are negatively affected by my gender"
,"I am aware of unpleasant and negative gossip in the workplace"
,"I am aware of abusive, disrespectful, or hostile treatment of employees"
,"I am aware of bullying behavior in the workplace"
,"I have been criticized for my personal communication style or appearance"
,"I have experienced or witnessed unwanted physical conduct in the workplace or by coworkers away from the workplace"
,"I have witnessed or heard of offensive or inappropriate jokes or comments in our workplace"
,"I am aware of sexually inappropriate phone calls, text messages, or social media attention from a coworker"
,"I have been the witness of conduct that I consider to be sexual harassment"
,"I feel unsafe at work because of inappropriate sexual conduct by some individuals"
,"I've seen career opportunities be favorably allocated on the basis of existing or expected sexual interactions"
,"I have considered leaving my job because of inappropriate gender-related behavior in the workplace"
,"I have experienced or witnessed inappropriate gender-related behavior by third parties (such as customers, vendors, and suppliers) associated with our organization"
,"Managers here tolerate or turn a blind eye to inappropriate sexual conduct"
,"I am aware of diminishing language toward colleagues based on their gender, disability and.or ethnic background"
,"I am aware of inappropriate and rude questions and comments regarding the pregnancy of my colleague.s"
,"I am aware of the inappropriate and rude questions and comments regarding the ethnic background of my colleague.s"
,"I am aware of the inappropriate and rude questions and comments regarding the disability of colleague.s"]]

# Melt the DataFrame to long format
melted_values = biases.melt(var_name='Statement', value_name='Rating')

# Count occurrences of each rating per statement
rating_counts_biases= melted_values.groupby(['Statement','Rating']).size().reset_index(name='Count')

In [30]:
'All of the time'=1,'Often'=2, 'Rarely'=3, 'Sometimes'=4	'Never'=5

,Statement,Rating,Count
0,"I am aware of abusive, disrespectful, or hosti...",All of the time,25
1,"I am aware of abusive, disrespectful, or hosti...",Never,20
2,"I am aware of abusive, disrespectful, or hosti...",Often,28
3,"I am aware of abusive, disrespectful, or hosti...",Rarely,19
4,"I am aware of abusive, disrespectful, or hosti...",Sometimes,9
...,...,...,...
85,Managers here tolerate or turn a blind eye to ...,All of the time,26
86,Managers here tolerate or turn a blind eye to ...,Never,17
87,Managers here tolerate or turn a blind eye to ...,Often,22
88,Managers here tolerate or turn a blind eye to ...,Rarely,19
